In [1]:
from __future__ import annotations
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.agents import AgentExecutor
from langchain_classic.agents import create_tool_calling_agent
import os
import shlex
import shutil
import subprocess
from pathlib import Path
from typing import Any, Dict, Optional
from langchain_openai import ChatOpenAI

In [ ]:
os.environ["OPENROUTER_API_KEY"] = ""


In [ ]:
# ----------------------------
# Guardrails / configuration
# ----------------------------
ALLOWED_DIR = Path(os.getenv("AGENT_DATA_DIR", ".")).resolve()
DEFAULT_TIMEOUT_S = int(os.getenv("AGENT_TIMEOUT_S", "30"))
os.environ["NUSMV_BIN"] = "./NuSMV"

NUSMV_BIN = os.getenv("NUSMV_BIN", "NuSMV")


def _resolve_safe_file(file_name_or_path: str) -> Path:
    """
    Resolve file path and restrict it to ALLOWED_DIR to prevent arbitrary file reads.
    """
    p = Path(file_name_or_path)
    p = (ALLOWED_DIR / p).resolve() if not p.is_absolute() else p.resolve()

    try:
        p.relative_to(ALLOWED_DIR)
    except ValueError as e:
        raise ValueError(f"File must be under allowed directory: {ALLOWED_DIR}") from e

    if not p.exists():
        raise FileNotFoundError(f"File not found: {p}")
    if not p.is_file():
        raise ValueError(f"Not a regular file: {p}")

    return p


def _find_nusmv_executable() -> str:
    """
    Prefer explicit path if provided; otherwise resolve via PATH.
    """
    if Path(NUSMV_BIN).is_absolute():
        if not Path(NUSMV_BIN).exists():
            raise FileNotFoundError(f"NuSMV not found at: {NUSMV_BIN}")
        return NUSMV_BIN

    resolved = shutil.which(NUSMV_BIN)
    if not resolved:
        raise FileNotFoundError(
            "NuSMV executable not found on PATH. "
            "Either add it to PATH or set environment variable NUSMV_BIN to its full path."
        )
    return resolved


@tool("run_nusmv")
def run_nusmv(
    smv_file: str = "short.smv",
    extra_args: Optional[str] = None,
    timeout_s: int = DEFAULT_TIMEOUT_S,
) -> Dict[str, Any]:
    """
    Run NuSMV on a given .smv file.

    Args:
      smv_file: Path (or filename) of the .smv input, restricted to AGENT_DATA_DIR.
      extra_args: Optional additional CLI args, e.g. "-dynamic" or "-dcx".
      timeout_s: Timeout in seconds.

    Returns:
      Structured result containing exit code, stdout, stderr.
    """
    exe = _find_nusmv_executable()
    safe_file = _resolve_safe_file(smv_file)

    cmd = [exe, str(safe_file)]
    if extra_args:
        cmd.extend(shlex.split(extra_args))

    try:
        p = subprocess.run(
            cmd,
            text=True,
            capture_output=True,
            timeout=timeout_s,
            check=False,
        )
        return {
            "command": cmd,
            "exit_code": p.returncode,
            "stdout": p.stdout[:20000],
            "stderr": p.stderr[:20000],
        }
    except subprocess.TimeoutExpired as e:
        return {
            "command": cmd,
            "exit_code": None,
            "stdout": (e.stdout or "")[:20000],
            "stderr": (e.stderr or "")[:20000],
            "error": f"Timed out after {timeout_s} seconds",
        }


TOOLS = [run_nusmv]

PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a verification assistant. "
            "When the user asks to analyze an SMV model, call the run_nusmv tool. "
            "Then interpret NuSMV output (counterexamples, satisfied/violated specs) and summarize clearly."
        ),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


def build_executor() -> AgentExecutor:
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        raise RuntimeError("Missing OPENROUTER_API_KEY environment variable.")


    llm = ChatOpenAI(
        model="mistralai/devstral-2512:free",
        api_key=api_key,
        base_url="https://openrouter.ai/api/v1",
        temperature=0.2,
    )

    agent = create_tool_calling_agent(llm=llm, tools=TOOLS, prompt=PROMPT)
    return AgentExecutor(
        agent=agent,
        tools=TOOLS,
        verbose=True,
        handle_parsing_errors=True,
    )

In [ ]:
print(f"Allowed dir: {ALLOWED_DIR}")
executor = build_executor()

result = executor.invoke(
    {
        "input": (
            "Run NuSMV on short.smv and summarize: "
            "which CTL/LTL specs pass or fail, and include any counterexample highlights."
        )
    }
)
print("\nASSISTANT:\n", result["output"])

Allowed dir: /Users/omersubasi/Downloads/AgentforModelChecking


> Entering new AgentExecutor chain...

Invoking: `run_nusmv` with `{'smv_file': 'short.smv', 'extra_args': None, 'timeout_s': 30}`


{'command': ['./NuSMV', '/Users/omersubasi/Downloads/AgentforModelChecking/short.smv'], 'exit_code': 0, 'stdout': '*** This is NuSMV 2.7.0 (compiled on Thu Oct 24 15:29:16 2024)\n*** Enabled addons are: compass\n*** For more information on NuSMV see <http://nusmv.fbk.eu>\n*** or email to <nusmv-users@list.fbk.eu>.\n*** Please report bugs to <Please report bugs to <nusmv-users@fbk.eu>>\n\n*** Copyright (c) 2010-2024, Fondazione Bruno Kessler\n\n*** This version of NuSMV is linked to the CUDD library version 2.4.1\n*** Copyright (c) 1995-2004, Regents of the University of Colorado\n\n*** This version of NuSMV is linked to the MiniSat SAT solver. \n*** See http://minisat.se/MiniSat.html\n*** Copyright (c) 2003-2006, Niklas Een, Niklas Sorensson\n*** Copyright (c) 2007-2010, Niklas Sorensson\n\n

In [5]:
print(f"Allowed dir: {ALLOWED_DIR}")
executor = build_executor()

result = executor.invoke(
    {
        "input": (
            "Run NuSMV on mutex.smv and summarize: "
            "which CTL/LTL specs pass or fail, and include any counterexample highlights."
        )
    }
)
print("\nASSISTANT:\n", result["output"])

Allowed dir: /Users/omersubasi/Downloads/AgentforModelChecking


> Entering new AgentExecutor chain...

Invoking: `run_nusmv` with `{'smv_file': 'mutex.smv', 'extra_args': None, 'timeout_s': 30}`


{'command': ['./NuSMV', '/Users/omersubasi/Downloads/AgentforModelChecking/mutex.smv'], 'exit_code': 0, 'stdout': '*** This is NuSMV 2.7.0 (compiled on Thu Oct 24 15:29:16 2024)\n*** Enabled addons are: compass\n*** For more information on NuSMV see <http://nusmv.fbk.eu>\n*** or email to <nusmv-users@list.fbk.eu>.\n*** Please report bugs to <Please report bugs to <nusmv-users@fbk.eu>>\n\n*** Copyright (c) 2010-2024, Fondazione Bruno Kessler\n\n*** This version of NuSMV is linked to the CUDD library version 2.4.1\n*** Copyright (c) 1995-2004, Regents of the University of Colorado\n\n*** This version of NuSMV is linked to the MiniSat SAT solver. \n*** See http://minisat.se/MiniSat.html\n*** Copyright (c) 2003-2006, Niklas Een, Niklas Sorensson\n*** Copyright (c) 2007-2010, Niklas Sorensson\n\n

# short.smv

MODULE main

VAR
  request : {Tr, Fa};

  state : {ready, busy};

ASSIGN

  init(state) := ready

  next(state) := case

      state = ready & (request = Tr): busy;

      TRUE : {ready,busy};

  esac;

SPEC

  AG((request = Tr) -> AF state = busy)


# mutex.smv

MODULE main

VAR

state1: {n1, t1, c1};

ASSIGN

init(state1) := n1;

next(state1) := 
case
   (state1 = n1) & (state2 = t2): t1;

   (state1 = n1) & (state2 = n2): t1;

   (state1 = n1) & (state2 = c2): t1;

   (state1 = t1) & (state2 = n2): c1;

   (state1 = t1) & (state2 = t2) & (turn = 1):  c1;

   (state1 = c1): n1;

   TRUE : state1;

esac;



VAR

state2: {n2, t2, c2};

ASSIGN

init(state2) := n2;

next(state2) := 
case
   (state2 = n2) & (state1 = t1): t2;

   (state2 = n2) & (state1 = n1): t2;

   (state2 = n2) & (state1 = c1): t2;

   (state2 = t2) & (state1 = n1): c2;

   (state2 = t2) & (state1 = t1) & (turn = 2):  c2;

   (state2 = c2): n2;

   TRUE : state2;
   
esac;


VAR

turn: {1, 2};

ASSIGN

init(turn) := 1;

next(turn) := 
case

   (state1 = n1) & (state2 = t2): 2;

   (state2 = n2) & (state1 = t1): 1;

   TRUE : turn;
   
esac;

SPEC

EF((state1 = c1) & (state2 = c2))

SPEC

AG((state1 = t1) -> AF (state1 = c1))

SPEC

AG((state2 = t2) -> AF (state2 = c2))